# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [26]:
weather_data = pd.read_csv("Output_Data/cities.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Blankenberge,16,BE,1603330338,81,51.31,3.13,57.00,3.00
1,Rawson,31,AR,1603330248,38,-43.30,-65.10,56.95,20.40
2,Yellowknife,75,CA,1603330290,79,62.46,-114.35,24.80,16.11
3,Hithadhoo,12,MV,1603330338,74,-0.60,73.08,82.60,12.88
4,Shingū,75,JP,1603330339,69,33.73,135.98,73.40,11.41
...,...,...,...,...,...,...,...,...,...
572,Coos Bay,1,US,1603330459,71,43.37,-124.22,55.99,17.22
573,Sinnamary,0,GF,1603330460,83,5.38,-52.95,79.75,9.75
574,Barra do Garças,92,BR,1603330460,78,-15.89,-52.26,75.52,4.18
575,Kenai,1,US,1603330387,41,60.55,-151.26,41.00,12.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
#configure gmaps
gmaps.configure(api_key=g_key)

#use lat and lgn as locations
locations = weather_data[["Lat", "Lng"]]

#use huimidity as weight
humidity = weather_data["Humidity"]

In [28]:
#plt the heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

#heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

#layers and layers!
fig.add_layer(heat_layer)

#print
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
#    Narrow down the cities to fit weather conditions.
new_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()
#   Drop any rows will null values.
new_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
124,Reconquista,0,AR,1603078896,66,-29.15,-59.65,73.74,8.79
126,Cayenne,0,GF,1603078897,100,4.93,-52.33,75.20,8.99
140,Erzin,0,TR,1603078899,87,36.96,36.20,71.01,1.48
154,Doka,0,SD,1603078903,78,13.52,35.77,71.82,3.94
175,Malindi,0,KE,1603078907,78,-3.22,40.12,77.94,6.49
176,Nuevo Progreso,0,MX,1603078908,94,23.17,-97.95,78.01,3.00
177,Bandarbeyla,0,SO,1603078908,73,9.49,50.81,79.81,3.76
253,Grand Gaube,0,MU,1603078924,65,-20.01,57.66,78.01,7.00
282,Young,0,UY,1603078930,82,-32.68,-57.63,70.02,8.14
285,Bilma,0,NE,1603078931,19,18.69,12.92,76.71,3.87


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
#Store into variable named hotel_df.
hotel_df = new_city_df.loc[:,["City", "Country", "Lat", "Lng"]]

#hotel df w. blank hotel name
hotel_df["Hotel Name"]=""

hotel_df

,City,Country,Lat,Lng,Hotel Name
124,Reconquista,AR,-29.15,-59.65,
126,Cayenne,GF,4.93,-52.33,
140,Erzin,TR,36.96,36.20,
154,Doka,SD,13.52,35.77,
175,Malindi,KE,-3.22,40.12,
176,Nuevo Progreso,MX,23.17,-97.95,
177,Bandarbeyla,SO,9.49,50.81,
253,Grand Gaube,MU,-20.01,57.66,
282,Young,UY,-32.68,-57.63,
285,Bilma,NE,18.69,12.92,


In [22]:
#URL and params

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"
    
    #api req
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    #results
    results = response['results']
    
    #hotel name in new df
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    #time sleep for api
    time.sleep(1)
    
#end print
print("------End of Search------")
    

Retrieving Results for Index 124: Reconquista.
Closest hotel in Reconquista is Grand Hotel.
------------
Retrieving Results for Index 126: Cayenne.
Closest hotel in Cayenne is Royal Amazonia.
------------
Retrieving Results for Index 140: Erzin.
Closest hotel in Erzin is Hattusa Vacation Thermal Club Erzin.
------------
Retrieving Results for Index 154: Doka.
Missing field/result... skipping.
------------
Retrieving Results for Index 175: Malindi.
Closest hotel in Malindi is Nyumba ya Mbuyu Boutique Hotel.
------------
Retrieving Results for Index 176: Nuevo Progreso.
Missing field/result... skipping.
------------
Retrieving Results for Index 177: Bandarbeyla.
Closest hotel in Bandarbeyla is Super secret pirate cove.
------------
Retrieving Results for Index 253: Grand Gaube.
Closest hotel in Grand Gaube is LUX Grand Gaube Resort & Villas.
------------
Retrieving Results for Index 282: Young.
Closest hotel in Young is Hotel Young.
------------
Retrieving Results for Index 285: Bilma.
M

In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))